In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
series = pd.read_csv('C:/mini project/Predict the minimum daily temperature/daily-minimum-temperatures-in-me.csv',
                header=0, index_col=0, parse_dates=True, squeeze=True, on_bad_lines='skip')
print(series)

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
              ... 
1990-12-27    14.0
1990-12-28    13.6
1990-12-29    13.5
1990-12-30    15.7
1990-12-31    13.0
Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, Length: 3650, dtype: object


In [3]:
#Extract Date-Time Features
df = pd.DataFrame()
df['month'] = [series.index[i].month for i in range(series.size)]
df['day'] = [series.index[i].day for i in range(series.size)]
df['temperature'] = [series[i] for i in range(series.size)]
df.head()

,month,day,temperature
0,1,1,20.7
1,1,2,17.9
2,1,3,18.8
3,1,4,14.6
4,1,5,15.8


In [4]:
#Lag Features
temp = pd.DataFrame(series.values)
df = pd.DataFrame(pd.concat([temp.shift(1), temp], axis=1))
df.columns = ['t', 't+1']
df.head()

,t,t+1
0,NaN,20.7
1,20.7,17.9
2,17.9,18.8
3,18.8,14.6
4,14.6,15.8


In [5]:
df = pd.DataFrame(pd.concat([temp.shift(3), temp.shift(2), temp.shift(1), temp], axis=1))
df.columns = ['t-2', 't-1', 't', 't+1']
df.head()

,t-2,t-1,t,t+1
0,NaN,NaN,NaN,20.7
1,NaN,NaN,20.7,17.9
2,NaN,20.7,17.9,18.8
3,20.7,17.9,18.8,14.6
4,17.9,18.8,14.6,15.8


In [6]:
#Rolling Window Statistics
shifted = temp.shift(1)
shifted = pd.DataFrame(pd.to_numeric(shifted.iloc[:,0].values, errors='coerce'))
shifted = shifted.astype('float32')
rolled = shifted.rolling(window=2)
feature = rolled.mean()
df = pd.DataFrame(pd.concat([feature, temp], axis=1))
df.columns = ['mean(t - 1, t)', 't + 1']
df.head()

,"mean(t - 1, t)",t + 1
0,NaN,20.7
1,NaN,17.9
2,19.300000,18.8
3,18.349999,14.6
4,16.700000,15.8


In [7]:

shifted = temp.shift(1)
shifted = pd.DataFrame(pd.to_numeric(shifted.iloc[:,0].values, errors='coerce'))
df.dropna(inplace=True)
shifted = shifted.astype('float32')
rolled = shifted.rolling(window=2)
df = pd.DataFrame(pd.concat([rolled.min(), rolled.max(), rolled.mean(), temp], axis=1))
df.columns = ['min', 'max', 'mean', 't + 1']
df.head()

,min,max,mean,t + 1
0,NaN,NaN,NaN,20.7
1,NaN,NaN,NaN,17.9
2,17.9,20.700001,19.300000,18.8
3,17.9,18.799999,18.349999,14.6
4,14.6,18.799999,16.700000,15.8


In [8]:
#Expanding Window Statistics
shifted = temp.shift(1)
shifted = pd.DataFrame(pd.to_numeric(shifted.iloc[:,0].values, errors='coerce'))
df.dropna(inplace=True)
shifted = shifted.astype('float32')
rolled = shifted.expanding()
df = pd.DataFrame(pd.concat([rolled.min(), rolled.max(), rolled.mean(), temp], axis=1))
df.columns = ['min', 'max', 'mean', 't + 1']
df.head()

,min,max,mean,t + 1
0,NaN,NaN,NaN,20.7
1,20.700001,20.700001,20.700001,17.9
2,17.900000,20.700001,19.300000,18.8
3,17.900000,20.700001,19.133333,14.6
4,14.600000,20.700001,18.000000,15.8
